In [2]:
import os
print(os.getcwd())

d:\OneDrive\Documents\ML Train\UNAL\Unidad 6\MLDS6_Proyecto\scripts\training


In [3]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from gensim.models import Word2Vec
import numpy as np

!ls
# Load data
data = pd.read_json('../../src/bazinga/database/Clean_Headlines.json', lines=True)
headlines = data['headline']
labels = data['is_sarcastic']
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26709 entries, 0 to 26708
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   article_link  26709 non-null  object
 1   headline      26709 non-null  object
 2   is_sarcastic  26709 non-null  int64 
 3   clean_text    26709 non-null  object
dtypes: int64(1), object(3)
memory usage: 834.8+ KB


'ls' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [5]:
# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

def get_bert_embeddings(texts, batch_size=32):
    
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Use [CLS] token and move back to CPU
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

bert_embeddings = get_bert_embeddings(headlines.tolist())

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [64]:
# Save BERT embeddings to file
np.save('bert_embeddings.npy', bert_embeddings)
bert_df = pd.DataFrame(bert_embeddings)
bert_df.to_csv('bert_embeddings.csv', index=False)

# Load BERT embeddings from file
loaded_bert_embeddings = np.load('bert_embeddings.npy')
loaded_bert_df = pd.read_csv('bert_embeddings.csv')
loaded_bert_embeddings_from_csv = loaded_bert_df.to_numpy()

In [6]:
print(bert_embeddings)

[[-0.27875116  0.12506711 -0.43516022 ... -0.26109472  0.1895398
   0.30300733]
 [ 0.28687197  0.1983599   0.06834903 ... -0.2625548   0.18669029
   0.46911165]
 [-0.54032004 -0.2303586   0.4740718  ... -0.4492011   0.36399528
   0.01749159]
 ...
 [-0.2593226  -0.09551121 -0.61755043 ... -0.05595841  0.46902105
   0.20650041]
 [-0.09609874  0.23279017 -0.24334535 ... -0.47870547 -0.01567071
   0.1109514 ]
 [-0.25170833  0.01995288  0.24350114 ... -0.31329003 -0.02248639
   0.32390368]]


In [7]:
# Tokenize the headlines
tokenized_headlines = [headline.split() for headline in headlines]

# Train Word2Vec model
w2v_model = Word2Vec(sentences=tokenized_headlines, vector_size=300, window=5, min_count=1, workers=4)

# Generate Word2Vec embeddings
def get_w2v_embeddings(texts, model):
    embeddings = []
    for text in texts:
        words = text.split()
        word_vectors = [model.wv[word] for word in words if word in model.wv]
        if word_vectors:
            embeddings.append(np.mean(word_vectors, axis=0))
        else:
            embeddings.append(np.zeros(300))  # Handle empty cases
    return np.array(embeddings)

w2v_embeddings = get_w2v_embeddings(headlines, w2v_model)

In [66]:
# Save Word2Vec embeddings
np.save('w2v_embeddings.npy', w2v_embeddings)
w2v_df = pd.DataFrame(w2v_embeddings)

# Save to CSV
w2v_df.to_csv('w2v_embeddings.csv', index=False)
loaded_w2v_embeddings = np.load('w2v_embeddings.npy')
loaded_w2v_df = pd.read_csv('w2v_embeddings.csv')
loaded_w2v_embeddings = loaded_w2v_df.to_numpy()

In [8]:
print(w2v_embeddings)

[[-0.02992626  0.21308674  0.03776343 ... -0.14672764  0.24013734
  -0.04816075]
 [-0.04379579  0.38479638  0.07330603 ... -0.2341625   0.40016124
  -0.09233301]
 [-0.03820257  0.37990794  0.07382133 ... -0.20238481  0.37978002
  -0.09466912]
 ...
 [-0.06332782  0.4649342   0.08560308 ... -0.3022044   0.50195813
  -0.10515881]
 [-0.01427451  0.08703323  0.01569336 ... -0.0582545   0.09946428
  -0.02333586]
 [-0.03153266  0.3034843   0.05635116 ... -0.19201021  0.31785488
  -0.07332972]]
